In [1]:
import os

root_paths = [
    "/data/kaggle-instacart",
    "/Users/jiayou/Dropbox/珺珺的程序/Kaggle/Instacart",
    "/Users/jiayou/Dropbox/Documents/珺珺的程序/Kaggle/Instacart"
]
root = None
for p in root_paths:
    if os.path.exists(p):
        root = p
        break
path = root
sbpath = os.path.join(root, 'sb')

In [2]:
import pandas as pd
import numpy as np

gorders = pd.read_csv(os.path.join(path, "orders.csv"), dtype={'order_id': np.uint32,
                                                              'user_id': np.uint32,
                                                              'eval_set': 'category',
                                                              'order_number': np.uint8,
                                                              'order_dow': np.uint8,
                                                              'order_hour_of_day': np.uint8
                                                              })

gorder_prior = pd.read_csv(os.path.join(path, "order_products__prior.csv"), dtype={'order_id': np.uint32,
                                                                                  'product_id': np.uint16,
                                                                                  'add_to_cart_order': np.uint8,
                                                                                  'reordered': bool})
gorder_prior = gorder_prior.merge(gorders[['order_id', 'user_id']], on='order_id', how='left')

gorder_train = pd.read_csv(os.path.join(path, "order_products__train.csv"), dtype={'order_id': np.uint32,
                                                                                  'product_id': np.uint16,
                                                                                  'add_to_cart_order': np.uint8,
                                                                                  'reordered': bool})
gorder_train = gorder_train.merge(gorders[['order_id', 'user_id']], on='order_id', how='left')

In [3]:
def load(shard=0, nshards=1000):
    global gorders, gorder_prior, gorder_train
    orders = gorders.loc[gorders.user_id % nshards == shard, :]
    order_prior = gorder_prior.loc[gorder_prior.user_id % nshards == shard, :]
    order_prior.drop('user_id', inplace = True, axis=1)
    order_train = gorder_train.loc[gorder_train.user_id % nshards == shard, :]
    order_train.drop('user_id', inplace = True, axis=1)
    return (orders, order_prior, order_train)

In [8]:
# user's product list

import gc
import pandas as pd
import numpy as np
import os

def create_products():
    orders, order_prior, order_train = load()

    print('loaded')

    orders = orders.loc[orders.eval_set == 'prior', :]
    orders_user = orders[['order_id', 'user_id']]
    labels = pd.merge(order_prior, orders_user, on='order_id')
    labels = labels.loc[:, ['user_id', 'product_id']].drop_duplicates()

    print('save')
    print(labels.shape)
    print(labels.columns)
    labels.to_pickle('sb/previous_products.pkl')
    return labels

In [9]:
labels = create_products()

loaded
save
(14046, 2)
Index(['user_id', 'product_id'], dtype='object')


/Users/jiayou/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jiayou/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
# abt skeleton without features

import gc
import pandas as pd
import numpy as np
import os

def split_data_set():
    path = root
    folds = 1

    aisles = pd.read_csv(os.path.join(path, "aisles.csv"), dtype={'aisle_id': np.uint8, 'aisle':'category'})
    departments = pd.read_csv(os.path.join(path, "departments.csv"), dtype={'department_id':np.uint8, 'department': 'category'})
    
    orders, order_prior, order_train = load()

    products = pd.read_csv(os.path.join(path, "products.csv"), dtype={'product_id': np.uint16,
                                                                      'aisle_id': np.uint8,
                                                                      'department_id': np.uint8})

    labels = pd.read_pickle('sb/previous_products.pkl')
    orders = orders.loc[(orders.eval_set == 'train') | (orders.eval_set == 'test'), :]
    labels = pd.merge(labels, orders[['order_id', 'user_id', 'eval_set']], on='user_id').drop(['user_id'], axis=1)

    order_train.drop(['add_to_cart_order'], axis=1, inplace=True)

    print('data is loaded')

    orders = np.unique(labels.order_id)

    size = orders.shape[0] // folds

    ret = []
    for fold in range(folds):

        current = orders[fold * size:(fold + 1) * size]

        current = labels.loc[np.in1d(labels.order_id, current), :]

        current = pd.merge(order_train, current, on=['order_id', 'product_id'], how='right')
        current.reordered.fillna(False, inplace=True)
        print(current.columns)
        print(current.shape)

        current.to_pickle('sb/chunk_{}.pkl'.format(fold))
        ret.append(current)
    return ret

In [11]:
folds = split_data_set()

/Users/jiayou/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jiayou/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jiayou/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


data is loaded
Index(['order_id', 'product_id', 'reordered', 'eval_set'], dtype='object')
(14046, 4)


In [12]:
# intervals a user purchases a product

import gc
import pandas as pd
import numpy as np
import os
import json
import sklearn.metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from scipy.sparse import dok_matrix, coo_matrix
from sklearn.utils.multiclass import  type_of_target

def orders_comsum():
    path = root
    sbpath = os.path.join(root, 'sb')

    aisles = pd.read_csv(os.path.join(path, "aisles.csv"), dtype={'aisle_id': np.uint8, 'aisle': 'category'})
    departments = pd.read_csv(os.path.join(path, "departments.csv"),
                              dtype={'department_id': np.uint8, 'department': 'category'})
    orders, order_prior, order_train = load()

    products = pd.read_csv(os.path.join(path, "products.csv"), dtype={'product_id': np.uint16,
                                                                      'aisle_id': np.uint8,
                                                                      'department_id': np.uint8})

    labels = pd.read_pickle(os.path.join(sbpath, 'chunk_0.pkl'))
    user_product = pd.read_pickle(os.path.join(sbpath, 'previous_products.pkl'))

    order_comsum = orders[['user_id', 'order_number', 'days_since_prior_order']].groupby(['user_id', 'order_number'])\
            ['days_since_prior_order'].sum().groupby(level=[0]).cumsum().reset_index().rename(columns={'days_since_prior_order':'days_since_prior_order_comsum'})

    # order_comsum['days_since_prior_order_comsum'].fillna(0, inplace=True)
    order_comsum.to_pickle('sb/orders_comsum.pkl')

    order_comsum2 = pd.merge(order_comsum, orders, on=['user_id', 'order_number'])[['user_id', 'order_number', 'days_since_prior_order_comsum', 'order_id']]

    order_product = pd.merge(order_prior, orders, on='order_id')[['order_id', 'product_id', 'eval_set']]
    order_product_train_test = labels[['order_id', 'product_id', 'eval_set']]

    order_product = pd.concat([order_product, order_product_train_test])

    order_product = pd.merge(order_product, order_comsum2, on='order_id')

    print(order_product.columns)

    order_product = pd.merge(order_product, user_product, on=['user_id', 'product_id'])

    temp = order_product.groupby(['user_id', 'product_id', 'order_number'])['days_since_prior_order_comsum'].sum().groupby(level=[0, 1]).apply(lambda x: np.diff(np.nan_to_num(x)))
    temp = temp.to_frame('periods').reset_index()

    temp.to_pickle('sb/product_period.pkl')

    aggregated = temp.copy()
    aggregated['last'] = aggregated.periods.apply(lambda x: x[-1])
    aggregated['prev1'] = aggregated.periods.apply(lambda x: x[-2] if len(x) > 1 else np.nan)
    aggregated['prev2'] = aggregated.periods.apply(lambda x: x[-3] if len(x) > 2 else np.nan)
    aggregated['median'] = aggregated.periods.apply(lambda x: np.median(x[:-1]))
    aggregated['mean'] = aggregated.periods.apply(lambda x: np.mean(x[:-1]))
    aggregated.drop('periods', axis=1, inplace=True)

    aggregated.to_pickle('sb/product_periods_stat.pkl')
    
    return (order_comsum, temp, aggregated)

In [13]:
oc = orders_comsum()

/Users/jiayou/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jiayou/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Index(['order_id', 'product_id', 'eval_set', 'user_id', 'order_number',
       'days_since_prior_order_comsum'],
      dtype='object')


/Users/jiayou/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [21]:
# user_dep_stat and user_aisle_stat

import gc
import pandas as pd
import numpy as np
import os
import json
import sklearn.metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from scipy.sparse import dok_matrix, coo_matrix
from sklearn.utils.multiclass import  type_of_target

def user_product_rank():
    path = root
    sbpath = os.path.join(root, 'sb')

    aisles = pd.read_csv(os.path.join(path, "aisles.csv"), dtype={'aisle_id': np.uint8, 'aisle': 'category'})
    departments = pd.read_csv(os.path.join(path, "departments.csv"),
                              dtype={'department_id': np.uint8, 'department': 'category'})

    orders, order_prior, order_train = load()

    products = pd.read_csv(os.path.join(path, "products.csv"), dtype={'product_id': np.uint16,
                                                                      'aisle_id': np.uint8,
                                                                      'department_id': np.uint8})

    order_train = pd.read_pickle(os.path.join(sbpath, 'chunk_0.pkl'))

    
    orders_products = pd.merge(orders, order_prior, on="order_id")

    orders_products_products = pd.merge(orders_products, products[['product_id', 'department_id', 'aisle_id']],
                                        on='product_id')

    user_dep_stat = orders_products_products.groupby(['user_id', 'department_id']).agg(
        {'product_id': lambda x: x.nunique(),
         'reordered': 'sum'
         })
    user_dep_stat.rename(columns={'product_id': 'dep_products',
                                  'reordered': 'dep_reordered'}, inplace=True)
    user_dep_stat.reset_index(inplace=True)
    print(user_dep_stat.columns)
    user_dep_stat.to_pickle('sb/user_department_products.pkl')

    user_aisle_stat = orders_products_products.groupby(['user_id', 'aisle_id']).agg(
        {'product_id': lambda x: x.nunique(),
         'reordered': 'sum'
         })
    user_aisle_stat.rename(columns={'product_id': 'aisle_products',
                                    'reordered': 'aisle_reordered'}, inplace=True)
    user_aisle_stat.reset_index(inplace=True)
    print(user_aisle_stat.columns)
    user_aisle_stat.to_pickle('sb/user_aisle_products.pkl')
    
    return (user_dep_stat, user_aisle_stat)

In [22]:
upr = user_product_rank()

/Users/jiayou/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jiayou/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Index(['user_id', 'department_id', 'dep_products', 'dep_reordered'], dtype='object')
Index(['user_id', 'aisle_id', 'aisle_products', 'aisle_reordered'], dtype='object')
